<a href="https://colab.research.google.com/github/Asmit-Dhakal/Deep_learning/blob/main/01_pytorch_workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **# Pytorch Work-flow**

In [2]:
topics={
    1:"data(prepare and load)",
    2: "build model",
    3: "fitting the model to the data(training)",
    4: "making prediciton and evaluating a model",
    5: "saving and loading model",
    6: "conclusion"
}

In [3]:
topics

{1: 'data(prepare and load)',
 2: 'build model',
 3: 'fitting the model to the data(training)',
 4: 'making prediciton and evaluating a model',
 5: 'saving and loading model',
 6: 'conclusion'}

In [5]:
import torch
from torch import nn # nn contain all building block for neural network
import matplotlib.pyplot as plt

# Check pytorch version
torch.__version__



'2.2.1+cu121'